In [ ]:
# Elif Zorlu, 10.11.2025

In [4]:
import random
import time
import copy
import math
from collections import defaultdict

In [5]:
def calculateNumberOfConflicts(board):
    """
    board: list of length N, board[i] = row of queen in column i
    returns total number of conflicting pairs (horizontal or diagonal)
    """
    numberOfConflicts = 0
    N = len(board)
    for i in range(N):
        for j in range(i+1, N):
            if board[i] == board[j]:
                numberOfConflicts += 1  # same row
            elif abs(i - j) == abs(board[i] - board[j]):
                numberOfConflicts += 1  # diagonal
    return numberOfConflicts


In [6]:
# this function places a queen per column in a random row
def generate_random_board(N):
  return [random.randrange(N) for _ in range(N)]

In [8]:
# this function returns a list of (neighbour_board, conflicts) for all possible single queen moves

def get_queen_neighbours(board):

  N = len(board)
  neighbours = []
  for column in range(N):
    for row in range(N):
      if row == board[column]:
        continue
      nb = board.copy()
      nb[column] = row
      neighbours.append((nb, calculateNumberOfConflicts(nb)))
  return neighbours

In [10]:
# this function applies hill clumbing. it returns (solution_board, success_bool, steps_taken)
# and  success_bool True if conflicts == 0


## Hill Climbing: - looks at ALL neighbours, picks the best one, moves there. However risk of getting
# stuck in a local minima.

def hill_climbing(start_board, max_steps = 10000):
  board = start_board.copy()
  steps = 0
  curr_conflicts = calculateNumberOfConflicts(board)
  N = len(board)
  while steps < max_steps:
        steps += 1
        neighbours = get_queen_neighbours(board)
        min_conflicts = min(conflicts for _, conflicts in neighbours)
        if min_conflicts >= curr_conflicts:
            break

        best = [nb for nb, conflicts in neighbours if conflicts == min_conflicts]
        board = random.choice(best)
        curr_conflicts = min_conflicts
        if curr_conflicts == 0:
            return board, True, steps
  return board, (curr_conflicts == 0), steps


In [11]:
# this function returns (found_board_or_last_board, success_bool, number_of_restarts, steps_total)

# Hill Climbing with Random Restarts: - same as basic hill climbing

def hill_climbing_random_restart(N, k, max_steps_per_run=10000):

  steps_total = 0
  for i in range(1, k+1):
    start = generate_random_board(N)
    sol, succ,steps = hill_climbing(start, max_steps=max_steps_per_run)
    steps_total += steps
    if succ:
      return sol, True, i, steps_total
  # if haven't found it using the amount of k steps
  return sol, False, k, steps_total


In [12]:
# Stochastic Hill Climbing:
# Looks at all neighbours, chooses from only the neighbours that are in a improving state,
# randomly chooses from one of the improving neighbours.

## - prevents the algorithm from getting stuck on paths that look steep at first but lead nowhere


def stochastic_hill_climbing(start_board, max_steps=10000):
    board = start_board.copy()
    steps = 0
    curr_conflicts = calculateNumberOfConflicts(board)

    while steps < max_steps:
        steps += 1
        neighbours = get_queen_neighbours(board)

        # filters neighbours so that only neighbours with lower conflicts are considered
        improving = [(nb, conf) for nb, conf in neighbours if conf < curr_conflicts]

        if not improving: # if reached a local minima
            break

        # uniform random choice among improving neighbours
        nb, conf = random.choice(improving)

        board = nb
        curr_conflicts = conf

        if curr_conflicts == 0:
            return board, True, steps

    return board, (curr_conflicts == 0), steps

In [16]:
def run_experiments_basic(N=20, runs=100):
    results = {"successes":0, "times":[] , "steps":[]}
    for _ in range(runs):
        start = generate_random_board(N)
        t0 = time.perf_counter()
        sol, succ, steps = hill_climbing(start)
        t1 = time.perf_counter()
        results["times"].append(t1-t0)
        results["steps"].append(steps)
        if succ:
            results["successes"] += 1
    avg_time = sum(results["times"]) / runs
    avg_steps = sum(results["steps"]) / runs
    return results["successes"], avg_time, avg_steps

def run_experiments_rr(N=20, runs=100, k=10):
    successes = 0
    times = []
    steps = []
    for _ in range(runs):
        t0 = time.perf_counter()
        sol, succ, used_restarts, total_steps = hill_climbing_random_restart(N, k)
        t1 = time.perf_counter()
        times.append(t1-t0)
        steps.append(total_steps)
        if succ:
            successes += 1
    return successes, sum(times)/runs, sum(steps)/runs


def run_experiments_stochastic(N=20, runs=100):
    successes = 0
    times = []
    steps = []

    for _ in range(runs):
        start = generate_random_board(N)
        t0 = time.perf_counter()
        sol, succ, s = stochastic_hill_climbing(start)
        t1 = time.perf_counter()

        times.append(t1 - t0)
        steps.append(s)
        if succ:
            successes += 1

    return successes, sum(times)/runs, sum(steps)/runs

In [17]:
N = 20
runs = 100

print("Running basic hill climbing...")
basic_res = run_experiments_basic(N=N, runs=runs)
print("Basic Hill Climbing:", basic_res)

print("Running hill climbing with random restart k=10...")
rr10 = run_experiments_rr(N=N, runs=runs, k=10)
print("RR k=10:", rr10)

print("Running hill climbing with random restart k=100...")
rr100 = run_experiments_rr(N=N, runs=runs, k=100)
print("RR k=100:", rr100)

print("Running stochastic hill climbing...")
stoch_uniform = run_experiments_stochastic(N=N, runs=runs)
print("Stochastic (uniform):", stoch_uniform)


Running basic hill climbing...
Basic: (5, 0.11451457536998987, 9.27)
Running random restart k=10...
RR k=10: (15, 1.0558414827899787, 86.61)
Running random restart k=100...
RR k=100: (83, 5.1894341134199795, 427.46)
Running stochastic hill climbing (uniform)...
Stochastic (uniform): (2, 0.18297969706000458, 15.85)
